In [1]:
from cryptography import x509

In [2]:
with open("google-com-chain.pem", "rb") as pem_file:
    certs = x509.load_pem_x509_certificates(pem_file.read())

In [3]:
assert len(certs) == 3 # Expect 3 certificates in the chain.

In [4]:
for idx, cert in enumerate(certs):
    print(f"cert {idx}, Subject: {cert.subject} Issuer: {cert.issuer}")

cert 0, Subject: <Name(CN=*.google.com)> Issuer: <Name(C=US,O=Google Trust Services,CN=WE2)>
cert 1, Subject: <Name(C=US,O=Google Trust Services,CN=WE2)> Issuer: <Name(C=US,O=Google Trust Services LLC,CN=GTS Root R4)>
cert 2, Subject: <Name(C=US,O=Google Trust Services LLC,CN=GTS Root R4)> Issuer: <Name(C=US,O=Google Trust Services LLC,CN=GTS Root R4)>
